In [ ]:
import datacube
from planetary_computer import sign_url
from dea_tools.plotting import rgb

In [ ]:
dc = datacube.Datacube(app='ls9-example')

In [ ]:
datasets = dc.find_datasets(product="ls9_c2l2_sr", region_code="069069", time="20230505")

print(f"Found {len(datasets)} datasets")

In [ ]:
data = dc.load(
    datasets=datasets,
    measurements=["red", "green", "blue"],
    output_crs="EPSG:3832",
    resolution=(-30, 30),
    lon=(-172.825, -172.125),
    lat=(-13.85, -13.4),
    patch_url=sign_url,
    skip_broken_datasets=True
)
data

In [ ]:
data = data.where(data != 0)
scaled = data * 2.75e-5 - 0.2

In [ ]:
rgb(scaled, bands=["red", "green", "blue"], size=10)

In [ ]:
import odc.ui
import ipyleaflet
from ipywidgets import widgets as w

polygons, bbox = odc.ui.dss_to_geojson(data.red, bbox=True)


zoom = odc.ui.zoom_from_bbox(bbox)
center = (bbox.bottom + bbox.top) * 0.5, (bbox.right + bbox.left) * 0.5

m = ipyleaflet.Map(
    center=center,
    zoom=zoom,
    scroll_wheel_zoom=True,  # Allow zoom with the mouse scroll wheel
    layout=w.Layout(
        width='600px',   # Set Width of the map to 600 pixels, examples: "100%", "5em", "300px"
        height='600px',  # Set height of the map
    ))

# Add full-screen and layer visibility controls
m.add_control(ipyleaflet.FullScreenControl())
m.add_control(ipyleaflet.LayersControl())

img_layer = odc.ui.mk_image_overlay(
    scaled.isel(time=0),
    clamp=(0, 0.2),
    # clamp=(2700, 20000),  # 3000 -- brightest pixel level
    fmt='png',
    bands=["red", "green", "blue"],
)   # "jpeg" is another option

# Add image layer to a map we created earlier
m.add_layer(img_layer)

slider = w.FloatSlider(min=0, max=1, value=1,        # Opacity is valid in [0,1] range
                       orientation='horizontal',     # Vertical slider is what we want
                       readout=False,                # No need to show exact value
                       layout=w.Layout(width='6em')) # Fine tune display layout: make it thinner

# Connect slider value to opacity property of the Image Layer
w.jslink((slider, 'value'),         
         (img_layer, 'opacity') )
m.add_control(ipyleaflet.WidgetControl(widget=slider))

display(m)